In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from utils import data_generator

IMAGENET_FOLDER = "val"

<h2> 2d bins </h2>

In [ ]:
# LAUNCH THIS BLOCK TO TRY OUT THE DATA GENERATOR
gen = data_generator(IMAGENET_FOLDER, 104)
feature, labels = next(gen)
plt.imshow(feature, cmap="gray")

In [ ]:
# LAUNCH THIS BLOCK TO VISUALIZE THE PRE PROCESS
from utils import pre_process, process_output

def plot_visualization(image):
    """
    Function to visualize the best output we could have from
    a perfect a model, given the information lost in the pre-process.
    (especially the resizing makes us lose a lot of details)
    """
    features, labels = pre_process(image, resolution=104)
    output = process_output(features, labels, np.shape(image))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("original image")
    
    plt.subplot(1, 3, 2)
    plt.imshow(output)
    plt.title("optimal network output")
    
    plt.subplot(1, 3, 3)
    plt.imshow(features, cmap="gray")
    plt.title("network input")


list_names = np.random.choice(next(os.walk(IMAGENET_FOLDER))[2], 6, replace=False)
for name in list_names:
    bgr_image = cv2.imread(IMAGENET_FOLDER + "/" + name)
    rgb_image = bgr_image[:, :, ::-1]
    
    plt.figure(figsize=(15, 5))
    plot_visualization(rgb_image)

In [ ]:
# PLOT HISTOGRAM OF LUMINANCE IN FEATURES (for 1,000 random images)
hist_dict = dict(((i, 0) for i in range(256)))
index = 0
for feature, label in gen:
    if index == 1000:
        break
    for value in feature.flatten():
        hist_dict[value] += 1
    index += 1
    
plt.bar(list(hist_dict.keys()), hist_dict.values())

<h2> regression input </h2>

In [ ]:
# LAUNCH THIS BLOCK TO TRY OUT THE DATA GENERATOR
gen = data_generator(IMAGENET_FOLDER, 104, is_regression=True)
feature, labels = next(gen)
plt.imshow(feature, cmap="gray")

In [ ]:
# LAUNCH THIS BLOCK TO VISUALIZE THE PRE PROCESS
from utils import pre_process_regression, process_output_regression

def plot_visualization(image):
    """
    Function to visualize the best output we could have from
    a perfect a model, given the information lost in the pre-process.
    (especially the resizing makes us lose a lot of details)
    """
    features, labels = pre_process_regression(image, resolution=104)
    output = process_output_regression(features, labels, np.shape(image))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("original image")
    
    plt.subplot(1, 3, 2)
    plt.imshow(output)
    plt.title("optimal network output")
    
    plt.subplot(1, 3, 3)
    plt.imshow(features, cmap="gray")
    plt.title("network input")


list_names = np.random.choice(next(os.walk(IMAGENET_FOLDER))[2], 6, replace=False)
for name in list_names:
    bgr_image = cv2.imread(IMAGENET_FOLDER + "/" + name)
    rgb_image = bgr_image[:, :, ::-1]
    
    plt.figure(figsize=(15, 5))
    plot_visualization(rgb_image)